https://huggingface.co/blog/inference-pro#getting-started-with-inference-for-pros

## Imports

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import os

HF_api_token = os.getenv("HF_API_KEY")

In [53]:
from huggingface_hub import InferenceClient

client = InferenceClient(model="meta-llama/Llama-2-70b-chat-hf", token=HF_api_token)

#for token in client.text_generation("Can you tell me all you know about string theory?", max_new_tokens=200, stream=True):
#    print(token)
output = client.text_generation("Can you tell me all you know about string theory?", max_new_tokens=200)
print(output)



I'm not sure I can tell you everything I know about string theory, as it is a very complex and technical subject. However, I can try to give you a brief overview of the main ideas and concepts.

String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. These two theories are the foundation of modern physics, but they are fundamentally incompatible within the framework of classical physics. Quantum mechanics describes the behavior of particles at the atomic and subatomic level, while general relativity describes the behavior of gravity and the large-scale structure of the universe.

The basic idea of string theory is that the fundamental building blocks of the universe are not point-like particles, but tiny, vibrating strings. These strings can vibrate at different frequencies, giving rise to the various particles we observe in the universe. In string theory, the properties of particles such as mass, charge, and


In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
from datasets import load_dataset

# Load the dataset from Hugging Face Datasets
dataset = load_dataset("fmuia/StringPheno", split="train")

base_model = 'meta-llama/Llama-2-70b-chat-hf'
finetuned_model = 'String-Pheno-Llama70b-model'

/Users/fmuia/miniforge3/envs/ai/lib/python3.8/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [4]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [5]:
push_to_hub = False
hf_token = HF_api_token
repo_id = "fmuia/StringPheno"
project_name = 'StringPheno_llm'

In [6]:
learning_rate = 2e-4
num_epochs = 4
batch_size = 1
block_size = 1024
trainer = "sft"
warmup_ratio = 0.1
weight_decay = 0.01
gradient_accumulation = 4
use_fp16 = True
use_peft = True
use_int4 = True
lora_r = 16
lora_alpha = 32
lora_dropout = 0.045

In [7]:
os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = finetuned_model
os.environ["PUSH_TO_HUB"] = str(push_to_hub)
os.environ["HF_TOKEN"] = hf_token
os.environ["REPO_ID"] = repo_id
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_EPOCHS"] = str(num_epochs)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["BLOCK_SIZE"] = str(block_size)
os.environ["WARMUP_RATIO"] = str(warmup_ratio)
os.environ["WEIGHT_DECAY"] = str(weight_decay)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["USE_FP16"] = str(use_fp16)
os.environ["USE_PEFT"] = str(use_peft)
os.environ["USE_INT4"] = str(use_int4)
os.environ["LORA_R"] = str(lora_r)
os.environ["LORA_ALPHA"] = str(lora_alpha)
os.environ["LORA_DROPOUT"] = str(lora_dropout)

In [9]:
autotrain llm \
--train \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--data-path data/ \
--text-column text \
--lr ${LEARNING_RATE} \
--batch-size ${BATCH_SIZE} \
--epochs ${NUM_EPOCHS} \
--block-size ${BLOCK_SIZE} \
--warmup-ratio ${WARMUP_RATIO} \
--lora-r ${LORA_R} \
--lora-alpha ${LORA_ALPHA} \
--lora-dropout ${LORA_DROPOUT} \
--weight-decay ${WEIGHT_DECAY} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
$( [[ "$USE_FP16" == "True" ]] && echo "--fp16" ) \
$( [[ "$USE_PEFT" == "True" ]] && echo "--use-peft" ) \
$( [[ "$USE_INT4" == "True" ]] && echo "--use-int4" ) \
$( [[ "$PUSH_TO_HUB" == "True" ]] && echo "--push-to-hub --token ${HF_TOKEN} --repo-id ${REPO_ID}" )

SyntaxError: invalid syntax (2552788404.py, line 1)

In [43]:
df = dataset["train"].to_pandas()
df['text'] = 'Context: String phenomenology is a field of study that explores the implications of string theory for particle physics and cosmology. ' + 'Question: ' + df['Question'] + ' Answer: ' + df['Answer']
df.head(3)

,Question,Answer,text
0,How do the bosonic generators in the component description of supersymmetry relate to the supersymmetry algebra?,"In the component description of supersymmetry, the bosonic generators are defined as $\delta_\xi = \xi Q + \bar{\xi} \bar{Q}$. The algebra defined with commutators of these generators is equivalent to the supersymmetry algebra. By calculating the commutators and ensuring that the algebra is represented on the component fields, one can demonstrate the consistency of the supersymmetry transformations at the component level.","Context: String phenomenology is a field of study that explores the implications of string theory for particle physics and cosmology. Question: How do the bosonic generators in the component description of supersymmetry relate to the supersymmetry algebra? Answer: In the component description of supersymmetry, the bosonic generators are defined as $\delta_\xi = \xi Q + \bar{\xi} \bar{Q}$. The algebra defined with commutators of these generators is equivalent to the supersymmetry algebra. By calculating the commutators and ensuring that the algebra is represented on the component fields, one can demonstrate the consistency of the supersymmetry transformations at the component level."
1,"How is the transition between different vacuum states, such as dS to AdS or dS to dS, estimated in the context of string theory compactifications as mentioned in the text?","The transition rates between different vacuum states, such as from de Sitter (dS) to Anti-de Sitter (AdS) or dS to dS, are estimated based on the potential for the volume modulus that connects these vacua. The probability amplitude for the transition, denoted as $g_s^7$ where $g_s^8$ is the value of the scalar potential at the dS minimum, reflects the likelihood of the transition occurring. The corresponding lifetime $g_s^9$ of such transitions is typically exponentially small compared to the Poincaré recurrence time, which is a reassuring feature.","Context: String phenomenology is a field of study that explores the implications of string theory for particle physics and cosmology. Question: How is the transition between different vacuum states, such as dS to AdS or dS to dS, estimated in the context of string theory compactifications as mentioned in the text? Answer: The transition rates between different vacuum states, such as from de Sitter (dS) to Anti-de Sitter (AdS) or dS to dS, are estimated based on the potential for the volume modulus that connects these vacua. The probability amplitude for the transition, denoted as $g_s^7$ where $g_s^8$ is the value of the scalar potential at the dS minimum, reflects the likelihood of the transition occurring. The corresponding lifetime $g_s^9$ of such transitions is typically exponentially small compared to the Poincaré recurrence time, which is a reassuring feature."
2,Can you explain how the spectra generated for density perturbations and gravitational waves during inflation provide a distinctive signature in the CMB anisotropies?,"The spectra of density perturbations and gravitational waves produced during inflation have distinct patterns based on the physics of inflation. These patterns leave characteristic imprints on the temperature fluctuations and polarization of the CMB. By analyzing the anisotropies in the CMB, we can study these spectra, which in turn provide valuable insights into the early universe, the nature of inflation, and the fundamental forces and particles at play during that time.","Context: String phenomenology is a field of study that explores the implications of string theory for particle physics and cosmology. Question: Can you explain how the spectra generated for density perturbations and gravitational waves during inflation provide a distinctive signature in the CMB anisotropies? Answer: The spectra of density perturbations and gravitational waves produced during inflation have distinct patterns based on the physics of inflation. These patterns l

In [44]:
QaAlist = df['text'].to_list()
QaAlist

['Context: String phenomenology is a field of study that explores the implications of string theory for particle physics and cosmology. Question: How do the bosonic generators in the component description of supersymmetry relate to the supersymmetry algebra? Answer: In the component description of supersymmetry, the bosonic generators are defined as $\\delta_\\xi = \\xi Q + \\bar{\\xi} \\bar{Q}$. The algebra defined with commutators of these generators is equivalent to the supersymmetry algebra. By calculating the commutators and ensuring that the algebra is represented on the component fields, one can demonstrate the consistency of the supersymmetry transformations at the component level.',
 'Context: String phenomenology is a field of study that explores the implications of string theory for particle physics and cosmology. Question: How is the transition between different vacuum states, such as dS to AdS or dS to dS, estimated in the context of string theory compactifications as ment

In [45]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [35]:
# Tokenize the Q&A list without converting to tensors directly
tokenized_inputs = tokenizer(QaAlist, truncation=True, max_length=512, padding=False)

In [40]:
from datasets import Dataset

# Assuming 'tokenized_inputs' is your dictionary containing tokenized data
data = Dataset.from_dict(tokenized_inputs)

In [41]:
# Initialize the data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)  # Optional: pad_to_multiple_of for efficient GPU usage

In [42]:
data_collator

DataCollatorWithPadding(tokenizer=LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-70b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=8, return_tensors='pt')

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [22]:
# Set the tokenizer's padding token to its EOS token if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Preprocess the dataset
def preprocess_function(examples):
    # Adjust these keys based on your dataset structure
    concatenated_qa = ["Question: " + q + " Answer: " + a for q, a in zip(examples['Question'], examples['Answer'])]
    return tokenizer(concatenated_qa, truncation=True)



Map:   0%|          | 0/2601 [00:00<?, ? examples/s]

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer', 'input_ids', 'attention_mask'],
        num_rows: 2601
    })
})

In [26]:
# Initialize the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
data_collator

DataCollatorWithPadding(tokenizer=LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-70b-chat-hf', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [14]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2601 [00:00<?, ? examples/s]

KeyError: 'text'



I'm not sure I can explain everything about string theory, as it is a highly complex and technical field of physics. However, I can provide a brief overview of the main ideas and concepts.

String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. These two theories are the foundation of modern physics, but they are fundamentally incompatible within the framework of classical physics. Quantum mechanics describes the behavior of particles at the atomic and subatomic level, while general relativity describes the behavior of gravity and the large-scale structure of the universe.

The fundamental idea of string theory is that the basic building blocks of the universe are not point-like particles, but tiny, vibrating strings. These strings can vibrate at different frequencies, giving rise to the various particles we observe in the universe. In string theory, the properties of particles such as mass, charge, and spin are determ